In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-22 14:42:55,913 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-22 14:42:55,915 - settings - Setting database.user to celiib
INFO - 2021-05-22 14:42:55,915 - settings - Setting database.password to newceliipass
INFO - 2021-05-22 14:42:55,964 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-22 14:42:55,965 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-22 14:42:55,979 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-22 14:42:56,464 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-22 14:42:56,504 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-22 14:42:56,505 - settings - Setting database.user to celiib
INFO - 2021-05-22 14:42:56,506 - settings - Setting database.password to newceliipass
INFO - 2021-05-22 14:42:56,507 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-22 14:42:56,508 - settings - Setting database.user to celiib
INFO - 2021-05-22 14:42:56,509 - settings - Setting database.password to newceliipass
INFO - 2021-05-22 14:42:56,512 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-22 14:42:57,036 - settings - Setting enable_python_native_blobs to True


In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = True

# Debugging the contains method

In [4]:
import system_utils as su

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-22 14:43:00,161 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-22 14:43:00,162 - settings - Setting database.user to celiib
INFO - 2021-05-22 14:43:00,163 - settings - Setting database.password to newceliipass
INFO - 2021-05-22 14:43:00,165 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-22 14:43:00,166 - settings - Setting database.user to celiib
INFO - 2021-05-22 14:43:00,167 - settings - Setting database.password to newceliipass
INFO - 2021-05-22 14:43:00,171 - settings - Setting enable_python_native_blobs to True


Sleeping 177 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-05-22 14:43:00,796 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 8478


In [8]:
# schema.external['decomposition'].delete(delete_external_files=True)

In [9]:
# key_source = minnie.Decomposition() & (minnie.AllenProofreading() & dict(month=3,day=18,year=2021)).proj()
# key_source

In [10]:
import numpy as np
import time
import proofreading_utils as pru

split_version = 0
split_version = 1 #fixed the problem with split from suggestions
split_version = 2 # adding all of the non-soma touching pieces to all of the splits

verbose = True

@schema
class DecompositionSplit(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index: tinyint unsigned  #the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    split_version: tinyint unsigned  #the version of the splitting algorithm used
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    n_splits             : int unsigned                 # the number of cuts required to help split the neuron
    split_success        : tinyint unsigned             # the successfulness of the splitting
    
    n_error_limbs_cancelled : tinyint unsigned     # number of limbs that couldn't be resolved and cancelled out        
    n_same_soma_limbs_cancelled : tinyint unsigned     # number of same soma touching limbs that couldn't be resolved and cancelled out
    n_multi_soma_limbs_cancelled : tinyint unsigned     # number of multi soma touching limbs that couldn't be resolved and cancelled out        
    
    error_imbs_cancelled_area=NULL : double            # the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas
    error_imbs_cancelled_skeletal_length = NULL : double #the total skeletal length (in microns) of the limbs that were called out because could not be resolved
    
    split_results: longblob #will store the results of how to split the limbs of neuron objects from original neuron
    decomposition: <decomposition>
    
    
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    max_soma_n_faces:  int unsigned                 # The largest number of faces of the somas
    max_soma_volume:  int unsigned                 # The largest volume of the somas the (volume in billions (10*9 nm^3))
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    """
                             
    
    ''' Old keysource used for inhibitory excitatory check
    classified_table = (minnie.BaylorManualCellType() &
                        'nucleus_version=3' & 
                        "(cell_type = 'excitatory') or  (cell_type = 'inhibitory')")
    
    key_source = ((minnie.Decomposition & 
                (minnie.NeuronSplitSuggestions.proj()) & 
                (classified_table.proj()) 
                & f"n_somas<{max_n_somas}" & "n_error_limbs>0"))'''
    
    # This keysource acounts that you could have more than 1 possible soma but not a significant limb connecting them (no error limbs)
    key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & (minnie.AllenProofreading() 
                                                                            & minnie.AllenProofreadingCurrentDate()).proj()
    
    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decomposed neurong object from Decomposition table
        3) Run the multi_soma split suggestions algorithm
        4) Get the number of splits required for this neuron
        5) Split the neuron into a list of neuron objects
        6) For each neuron object in the list:
        - get the number of errored limbs (to indicate the success type)
        - Change the description to include the multiplicity
        - Compute the information on the largest soma faces and volume
        - Save the neuron object to the external
        - Add the new write key to a list to commit 
        7) Write all of the keys 
        """
        
        whole_pass_time = time.time()
        
        
        
        # 1) Get the segment id from the key
        segment_id = key["segment_id"]
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        
        
        # 2) Get the decomposed neuron object from Decomposition table and the split suggestions
        neuron_obj_path = (minnie.Decomposition & key).fetch1("decomposition")
        neuron_obj = du.filepath_to_neuron_obj(neuron_obj_path)
        
        """ Old way that downloaded from another table
        # 3) Retrieve the multi soma suggestions
        split_results = (minnie.NeuronSplitSuggestions & key).fetch1("split_results")
        """
        #3) Calculated the split results
        split_results = pru.multi_soma_split_suggestions(neuron_obj,plot_intermediates=False)
        
        # 4) Get the number of splits required for this neuron
        n_paths_cut = pru.get_n_paths_cut(split_results)

        if verbose:
            print(f"n_paths_cut = {n_paths_cut}")
            
            
        # 5) Split the neuron into a list of neuron objects
        (neuron_list,
        neuron_list_errored_limbs_area,
         neuron_list_errored_limbs_skeletal_length,
        neuron_list_n_multi_soma_errors,
        neuron_list_n_same_soma_errors) = pru.split_neuron(neuron_obj,
                        limb_results=split_results,
                                       verbose=verbose,
                                        return_error_info=True
                                            )
        
        print(f"neuron_list = {neuron_list}")
        print(f"neuron_list_errored_limbs_area = {neuron_list_errored_limbs_area}")
        print(f"neuron_list_n_multi_soma_errors = {neuron_list_n_multi_soma_errors}")
        print(f"neuron_list_n_same_soma_errors = {neuron_list_n_same_soma_errors}")
        
        
        if verbose:
            print(f"Number of neurons: {len(neuron_list)}")
        
        neuron_entries = []
        for neuron_idx in range(len(neuron_list)):
            
            """
            # 6) For each neuron object in the list:
            # - get the number of errored limbs (to indicate the success type)
            # - Compute the information on the largest soma faces and volume
            # - Save the neuron object to the external
            # - Add the new write key to a list to commit 
            """
            n = neuron_list[neuron_idx]
            
            error_imbs_cancelled_area = neuron_list_errored_limbs_area[neuron_idx]
            error_imbs_cancelled_skeletal_length = neuron_list_errored_limbs_skeletal_length[neuron_idx]
            n_multi_soma_limbs_cancelled = neuron_list_n_multi_soma_errors[neuron_idx]
            n_same_soma_limbs_cancelled = neuron_list_n_same_soma_errors[neuron_idx]
            
            
            #for n in neuron_list:
            #     nviz.visualize_neuron(n,
            #                          limb_branch_dict="all")

            # - get the number of errored limbs (to indicate the success type)
            if n.n_error_limbs == 0:
                split_success = 0
            elif n.multi_soma_touching_limbs == 0:
                split_successs = 1
            elif n.same_soma_multi_touching_limbs == 0:
                split_success = 2
            else:
                split_success = 3

            if verbose:
                print(f"split_success = {split_success}")

            # - Compute the information on the largest soma faces and volume
            soma_volumes = [n[k].volume/1000000000 for k in n.get_soma_node_names()] 
            soma_n_faces = [len(n[k].mesh.faces) for k in n.get_soma_node_names()] 

            largest_n_faces = np.max(soma_n_faces)
            largest_volume = np.max(soma_volumes)

            if verbose:
                print(f"largest_n_faces = {largest_n_faces}")
                print(f"largest_volume = {largest_volume}")

            if "split" not in n.description:
                n.description += "_soma_0_split"
                
            #6) Save the file in a certain location
            if True:
                save_time = time.time()
                ret_file_path = n.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                                  return_file_path=True,
                                                 export_mesh=False,
                                                 suppress_output=True)

                ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
                print(f"Save time = {time.time() - save_time}")
            else:
                print("Storing a dummy value for neuron")
                ret_file_path_str = "dummy"



            #7) Pass stats and file location to insert
            new_key = dict(key,
                           split_index = neuron_idx,
                           split_version = split_version,
                           
                           multiplicity=len(neuron_list),

                           n_splits = n_paths_cut,
                           split_success = split_success,
                           
                           n_error_limbs_cancelled = len(error_imbs_cancelled_area),
                           
                           n_multi_soma_limbs_cancelled =n_multi_soma_limbs_cancelled,
                           n_same_soma_limbs_cancelled = n_same_soma_limbs_cancelled,
                           error_imbs_cancelled_area = np.round(np.sum(error_imbs_cancelled_area),4),
                           error_imbs_cancelled_skeletal_length = np.round(np.sum(error_imbs_cancelled_skeletal_length)/1000,4),
                           
                           split_results=split_results,

                           max_soma_n_faces = largest_n_faces,
                           max_soma_volume = largest_volume,


                           decomposition=ret_file_path_str,
                           n_vertices=len(n.mesh.vertices),
                           n_faces=len(n.mesh.faces),
                           run_time=np.round(time.time() - whole_pass_time,4)
                          )

            stats_dict = n.neuron_stats()
            new_key.update(stats_dict)
            
            attributes_to_remove = ["axon_length","axon_area","n_boutons"]
            
            for k in attributes_to_remove:
                del new_key[k]
            
            neuron_entries.append(new_key)

        
        self.insert(neuron_entries, allow_direct_insert=True, skip_duplicates=True)
        
        print(f"\n\n ------ Total time for {segment_id} = {time.time() - whole_pass_time} ------")
    

# Running the Populate

In [15]:
# du.restrict_jobs_table_by_error_substring(curr_table,"KeyboardInterrupt").delete()

In [12]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition_split,5e8b61293856de161f7df946e85e2299,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.145,at-node18,1,444416,2021-05-21 15:46:01
__decomposition_split,66499ab7ce9027faac7458a2c3342b82,error,=BLOB=,KeyboardInterrupt,=BLOB=,celiib@10.28.0.83,at-compute004,167581,446861,2021-05-22 09:43:31
__decomposition_split,9e882857c8997633d321de790e884319,error,=BLOB=,IndexError: list index out of range,=BLOB=,celiib@10.28.0.146,at-node19,1,444398,2021-05-21 15:10:47
__decomposition_split,f90ef88483b19f9b693412c09804c250,error,=BLOB=,IndexError: list index out of range,=BLOB=,celiib@10.28.0.165,at-node38,1,444422,2021-05-21 15:29:13


ERROR! Session/line number was not unique in database. History logging moved to new session 715


In [ ]:
import time
import random
pru = reload(pru)
nru = reload(nru)
import neuron
neuron = reload(neuron)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=True)
else:
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionSplit populate = {time.time() - start_time}")

INFO - 2021-05-22 14:43:12,445 - autopopulate - Found 4 keys to populate
INFO - 2021-05-22 14:43:12,461 - connection - Transaction started
INFO - 2021-05-22 14:43:12,463 - autopopulate - Populating: {'segment_id': 864691135866531094, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('30.00'), 'process_version': 7, 'index': 0}


Populate Started



---- Working on Neuron 864691135866531094 ----


INFO - 2021-05-22 14:43:31,042 - connection - Transaction cancelled. Rolling back ...
ERROR - 2021-05-22 14:43:31,336 - ultratb - Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO - 2021-05-22 14:43:31,344 - ultratb - 
Unfortunately, your original traceback can not be constructed.

ERROR - 2021-05-22 14:43:31,447 - ultratb - Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO - 2021-05-22 14:43:31,458 - ultratb - 
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-a82ef80a41f6>", line 15, in <module>
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=False)
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "<ipython-input-10-60ac8e861627>", line 122, in make
    neuron_obj = du.filepath_to_neuron_obj(neuron_obj_path)
  File "/meshAfterParty/meshAfterParty/datajoint_utils.py", line 1237, in filepath_to_neuron_obj
    dec_mesh = fetch_segment_id_mesh(segment_id)
  File "/meshAfterParty/meshAfterParty/datajoint_utils.py", line 750, in fetch_segment_id_mesh
    new_mesh = (minnie.Decimation() & key).fetch1("mesh")
  File "/usr/local/lib/python3.6/dist-packages/datajoint/fetch.py", line 245, in __call__
    result = self._expression.proj(*attribu

ERROR - 2021-05-22 14:43:31,621 - ultratb - Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO - 2021-05-22 14:43:31,628 - ultratb - 
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-a82ef80a41f6>", line 15, in <module>
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=False)
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "<ipython-input-10-60ac8e861627>", line 122, in make
    neuron_obj = du.filepath_to_neuron_obj(neuron_obj_path)
  File "/meshAfterParty/meshAfterParty/datajoint_utils.py", line 1237, in filepath_to_neuron_obj
    dec_mesh = fetch_segment_id_mesh(segment_id)
  File "/meshAfterParty/meshAfterParty/datajoint_utils.py", line 750, in fetch_segment_id_mesh
    new_mesh = (minnie.Decimation() & key).fetch1("mesh")
  File "/usr/local/lib/python3.6/dist-packages/datajoint/fetch.py", line 245, in __call__
    result = self._expression.proj(*attribu